In [1]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});

<IPython.core.display.Javascript object>

In [23]:
import sys; sys.argv=['']; del sys
import os
import time
import numpy as np
import tensorflow as tf

import data_helpers
from configure import FLAGS
from logger import Logger
from model.self_lstm import SelfLSTM
from model.self_cnn import SelfCNN
import utils

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)


from tensor2tensor.visualization import attention

FLAGS.checkpoint_dir = "runs/lstm-84.34/checkpoints"
with tf.device('/cpu:0'):
    test_text, test_y, test_pos1, test_pos2 = data_helpers.load_data_and_labels(FLAGS.test_path)

checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
print(checkpoint_file)

vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocab_path)

# Map data into position
position_path = os.path.join(FLAGS.checkpoint_dir, "..", "pos_vocab")
pos_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(position_path)

test_x = np.array(list(vocab_processor.transform(test_text)))
test_text = np.array(test_text)
print("\nText Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("test_x = {0}".format(test_x.shape))
print("test_y = {0}".format(test_y.shape))

test_p1 = np.array(list(pos_vocab_processor.transform(test_pos1)))
test_p2 = np.array(list(pos_vocab_processor.transform(test_pos2)))
print("\nPosition Vocabulary Size: {:d}".format(len(pos_vocab_processor.vocabulary_)))
print("test_p1 = {0}".format(test_p1.shape))
print("")

output_alphas = []

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=FLAGS.allow_soft_placement,
        log_device_placement=FLAGS.log_device_placement)
    session_conf.gpu_options.allow_growth = FLAGS.gpu_allow_growth
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        input_x = graph.get_operation_by_name("input_x").outputs[0]
        input_y = graph.get_operation_by_name("input_y").outputs[0]
        input_p1 = graph.get_operation_by_name("input_p1").outputs[0]
        input_p2 = graph.get_operation_by_name("input_p2").outputs[0]
        emb_dropout_keep_prob = graph.get_operation_by_name("emb_dropout_keep_prob").outputs[0]
        rnn_dropout_keep_prob = graph.get_operation_by_name("rnn_dropout_keep_prob").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]
        self_alphas_op = graph.get_operation_by_name("self-attention/multihead_attention/Softmax").outputs[0]
        alphas_op = graph.get_tensor_by_name("attention/alphas:0")
        acc_op = graph.get_tensor_by_name("accuracy/accuracy:0")

        print("\nEvaluation:")
        # Generate batches
        test_batches = data_helpers.batch_iter(list(zip(test_x, test_y, test_p1, test_p2)),
                                               FLAGS.batch_size, 1, shuffle=False)
        # Training loop. For each batch...
        accuracy = 0.0
        iter_cnt = 0
        for test_batch in test_batches:
            test_bx, test_by, test_bp1, test_bp2 = zip(*test_batch)
            feed_dict = {
                input_x: test_bx,
                input_y: test_by,
                input_p1: test_bp1,
                input_p2: test_bp2,
                emb_dropout_keep_prob: 1.0,
                rnn_dropout_keep_prob: 1.0,
                dropout_keep_prob: 1.0
            }
            self_alphas, alphas, acc = sess.run(
                [self_alphas_op, alphas_op, acc_op], feed_dict)
            accuracy += acc
            self_alphas = np.reshape(self_alphas, (FLAGS.num_heads, -1, FLAGS.max_sentence_length, FLAGS.max_sentence_length))
            self_alphas = self_alphas.transpose([1, 0, 2, 3])
            
            output_alphas += self_alphas.tolist()
            
            


SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
max sentence length = 68

/home/joohong/PycharmProjects/self-attention-relation-extraction/runs/lstm-84.34/checkpoints/model-83.9-3600

Text Vocabulary Size: 22384
test_x = (2717, 90)
test_y = (2717, 19)

Position Vocabulary Size: 162
test_p1 = (2717, 90)

INFO:tensorflow:Restoring parameters from /home/joohong/PycharmProjects/self-attention-relation-extraction/runs/lstm-84.34/checkpoints/model-83.9-3600

Evaluation:


In [107]:
idx = 2551  # 72  # 218 1130 1258 2551

self_alphas = np.array(output_alphas[idx])
sent = test_text[idx].split()
sent_len = len(sent)
s = [self_alphas[0][:sent_len, :sent_len]*34]
s.append(self_alphas[1][:sent_len, :sent_len]*30)
s.append(self_alphas[2][:sent_len, :sent_len]*29)
s.append(self_alphas[3][:sent_len, :sent_len]*29)

for i in range(len(s)):
    for j in range(len(s[i])):
        for k in range(len(s[i][j])):
            if(s[i][j][k]<0.48):
                s[i][j][k]= 0.


s = [np.array(s)]

attention.show(sent, sent, s, s, s)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>